# Obtaining and Plotting NEXRAD Radar Data to Create Animations

In [ ]:
from pylab import *
import pyart, boto3, tempfile, os, shutil, datetime, matplotlib
import numpy as np
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from matplotlib import animation
from botocore.handlers import disable_signing
from glob import glob

In [ ]:
def get_radar_scan(station='KLOT', date=None, key_index=-20):
    
    '''
    Function will pull the latest radar scan from any radar site using 
    Amazon S3.
    ----------
    Station = Four letter NEXRAD identifier
              Example: 'KEPZ'
    Date = default is none for current date, else enter date in format "YYYY/MM/DD"
    Ex: date ='2013/11/17
    Key_index = Number of keys you want pulled from most recent scan.
    Ex: key_index = -15 would pull ht most recent 15 scans
    '''
    
    # Creating a bucket and a client to be able to pull data from AWS and setting it as unsigned
    bucket = 'noaa-nexrad-level2'
    s3 = boto3.resource('s3')
    s3.meta.client.meta.events.register('choose-signer.s3.*', disable_signing)
    
    # Connects the bucket create above with radar data
    aws_radar = s3.Bucket(bucket)
    
    # Setting the date and time to current...
    # This will allow for allow the current date's radar scands to be pulled
    if date == None:
        target_string = datetime.datetime.utcnow().strftime('%Y/%m/%d/'+station)
    else:
        target_string = date+'/'+station
    
    for obj in aws_radar.objects.filter(Prefix= target_string):
        '{0}:{1}'.format(aws_radar.name, obj.key)
    my_list_of_keys = [this_object.key for this_object in aws_radar.objects.filter(Prefix= target_string)]
    keys = my_list_of_keys[key_index:]
    for key in keys:
        if 'MDM' in key:
            keys.remove(key)
    print(keys)
    return aws_radar, keys

In [ ]:
# Setting radar, date of radar scans needed, and key index (amount of items in list)
aws_radar, keys = get_radar_scan(station='KLOT', date='2013/04/18', key_index=-400) 
out_path_dir = 'home/amedendorp/Desktop/april182013'

In [ ]:
# Obtaining every 7th step (approximately every half hour). 
# Leave blank bracket to include all timesteps
newkeys=keys[0:600:7]
print(newkeys)

In [ ]:
def radar_keys_to_data(newkeys):
    # Stores keys into temporary localfile to be read with pyart. 
    localfile = tempfile.NamedTemporaryFile()
    keys.get_contents_to_filename(localfile.name)
    # Only pulling two scans for the sake of time and memory.
    radar = pyart.io.read_nexrad_archive(localfile.name)
    return radar

In [ ]:
import cartopy.io.shapereader as shpreader

reader = shpreader.Reader('/home/amedendorp/Downloads/countyl010g_shp_nt00964/countyl010g.shp')
counties = list(reader.geometries())
COUNTIES = cfeature.ShapelyFeature(counties, ccrs.PlateCarree())

In [ ]:
# Animation code:
def animate(nframe):
    plt.clf()
    localfile = tempfile.NamedTemporaryFile()
    aws_radar.download_file(newkeys[nframe], localfile.name)
    radar = pyart.io.read(localfile.name)
    ax = plt.axes(projection=ccrs.PlateCarree())
    display = pyart.graph.RadarMapDisplay(radar)
    
    # Delete radar after use to save memory.
    del radar
    display.plot_ppi_map('reflectivity', sweep=0, resolution='10m',
                         vmin=-8, vmax=64, mask_outside=False, 
                         fig=fig, height=200000, width=150000) 

    display.plot_point(-87.981810, 41.713969, label_text='ANL', symbol='ko')
    display.plot_point(-87.675885, 42.057888, label_text='Northwestern', symbol='ko')
    display.plot_point(-88.181798, 41.738107, label_text='NPR', symbol='ko') #Naperville
    display.plot_point(-87.687151, 41.606367, label_text='IBP', symbol='ko')
    
    ax.add_feature(cfeature.LAKES, zorder=0.5)
    ax.add_feature(COUNTIES, facecolor='none', edgecolor='gray')

fig = plt.figure(figsize=(10, 8))

anim_klot = animation.FuncAnimation(fig, animate,
                                    frames=len(newkeys))
anim_klot.save('/home/amedendorp/Desktop/reflectivity_animation_three.gif', 
               writer='imagemagick', fps=3)
plt.show()
plt.close()